<a href="https://colab.research.google.com/github/rondinell/Intelig-ncia-Artificial/blob/main/Artigo16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================================================
# Artigo 16: O Poder da Atenção - Usando Transformers para Análise do Mercado Financeiro
# Autor: Rondinelli Alves de Andrade, com assistência de IA
# Data: 11 de Agosto de 2025
#
# Descrição: Este código implementa um modelo de Deep Learning baseado na
# arquitetura Transformer para prever a tendência de ativos financeiros.
# Diferente das LSTMs que processam dados sequencialmente, os Transformers
# usam um mecanismo de "auto-atenção" para pesar a importância de
# diferentes pontos no tempo de uma só vez, capturando relações complexas
# de longo prazo de forma mais eficaz, como são três pontos de atenção, baixa, alta, neutro.
# =========================================================================

# ===========================================================
# Parte 1: INSTALAÇÃO E IMPORTAÇÕES
# ===========================================================
print("Instalando e importando bibliotecas...")
!pip install yfinance --quiet
import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import date

print("Bibliotecas carregadas com sucesso!")

# =========================================================
# Parte 2: CONFIGURAÇÕES GLOBAIS
# =========================================================
print("\nDefinindo configurações globais.")

# Defina o ticker do ativo que deseja analisar
TICKER = 'PETR4.SA' # Exemplo: Petrobras

START_DATE = '2015-01-01'
END_DATE = date.today().strftime("%Y-%m-%d")

# --- Parâmetros da Sequência e do Alvo ---
SEQ_LENGTH = 60      # Usar os últimos 60 dias de dados para prever
FUTURE_DAYS = 5      # Tentar prever a tendência 5 dias no futuro
THRESHOLD = 0.035    # Variação de 3.5% para definir ALTA ou BAIXA

# --- Parâmetros do Modelo Transformer ---
HEAD_SIZE = 256      # Dimensionalidade da "cabeça" de atenção
NUM_HEADS = 4        # Número de "cabeças" de atenção (para multi-head attention)
FF_DIM = 4           # Dimensão da camada Feed-Forward interna do bloco
NUM_BLOCKS = 4       # Número de blocos Transformer empilhados
DROPOUT = 0.2        # Taxa de dropout para regularização

# --- Parâmetros de Treinamento ---
BATCH_SIZE = 32
EPOCHS = 50 # Transformers podem precisar de mais épocas para convergir

print(f"Configurações definidas para o ativo: {TICKER}")

# ======================================================
# Parte 3: DOWNLOAD E CÁLCULO DE INDICADORES
# ======================================================
print(f"\nBaixando dados históricos para {TICKER}...")
data = yf.download(TICKER, start=START_DATE, end=END_DATE, auto_adjust=True, progress=False)

# --- Cálculo de Indicadores Técnicos via Pandas ---
print("Calculando indicadores técnicos...")
# Bandas de Bollinger
window_bb = 20
data['bb_ma'] = data['Close'].rolling(window=window_bb).mean()
data['bb_std'] = data['Close'].rolling(window=window_bb).std()
data['bb_high'] = data['bb_ma'] + (data['bb_std'] * 2)
data['bb_low'] = data['bb_ma'] - (data['bb_std'] * 2)

# RSI (Índice de Força Relativa)
window_rsi = 14
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).ewm(com=window_rsi - 1, adjust=False).mean()
loss = (-delta.where(delta < 0, 0)).ewm(com=window_rsi - 1, adjust=False).mean()
rs = gain / loss
data['rsi'] = 100 - (100 / (1 + rs))

data.dropna(inplace=True)
print(f"Dados processados. Total de {len(data)} registros.")

# ============================================================
# Parte 4: PREPARAÇÃO DOS DADOS PARA O TRANSFORMER
# ============================================================
print("\nPreparando sequências e rótulos para o modelo...")

# Definindo as features (características) que o modelo usará
features = ['Close', 'Volume', 'bb_high', 'bb_low', 'rsi']
data_featured = data[features]

# Normalizando as features para a escala [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data_featured)

# --- Geração das sequências e dos rótulos ---
sequences, labels = [], []
class_names = ['ALTA', 'BAIXA', 'NEUTRA']
class_map = {name: i for i, name in enumerate(class_names)}

for i in range(len(data_scaled) - SEQ_LENGTH - FUTURE_DAYS):
    # Pega uma sequência de 60 dias de dados normalizados
    sequences.append(data_scaled[i: i + SEQ_LENGTH])

    # Pega o preço de fechamento no final da janela e o preço futuro para definir o rótulo
    preco_final_janela = data['Close'].iloc[i + SEQ_LENGTH - 1]
    preco_futuro = data['Close'].iloc[i + SEQ_LENGTH + FUTURE_DAYS - 1]

    # Define o rótulo com base na variação percentual
    # Extract scalar values for comparison
    if preco_futuro.item() > preco_final_janela.item() * (1 + THRESHOLD):
        labels.append(class_map['ALTA'])
    elif preco_futuro.item() < preco_final_janela.item() * (1 - THRESHOLD):
        labels.append(class_map['BAIXA'])
    else:
        labels.append(class_map['NEUTRA'])


X = np.array(sequences)
y = np.array(labels)

# --- Divisão em Conjuntos de Treino e Validação ---
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Total de {len(X)} exemplos criados.")
print(f"Dados de Treino: {len(X_train)} sequências")
print(f"Dados de Validação: {len(X_val)} sequências")

# ============================================================
# Parte 5: CONSTRUÇÃO DO MODELO TRANSFORMER
# ============================================================
print("\nConstruindo a arquitetura do Transformer...")

def transformer_encoder(inputs):
    # --- Bloco de Atenção e Normalização ---
    # O MultiHeadAttention é o coração do Transformer. Ele permite que o modelo
    # olhe para diferentes partes da sequência ao mesmo tempo ("múltiplas cabeças")
    # para aprender relações complexas.
    attention = layers.MultiHeadAttention(
        key_dim=HEAD_SIZE, num_heads=NUM_HEADS, dropout=DROPOUT
    )(inputs, inputs)

    # Conexão Residual (soma a entrada original) para evitar o problema do
    # desaparecimento do gradiente.
    attention = layers.Dropout(DROPOUT)(attention)
    attention = layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    # --- Bloco Feed-Forward ---
    # Uma rede neural simples que processa a saída da camada de atenção.
    ffn = keras.Sequential([
        layers.Dense(FF_DIM, activation="relu"),
        layers.Dense(inputs.shape[-1]), # Projeta de volta para a dimensão original
    ])
    outputs = ffn(attention)

    # Segunda Conexão Residual e Normalização
    outputs = layers.Dropout(DROPOUT)(outputs)
    outputs = layers.LayerNormalization(epsilon=1e-6)(attention + outputs)

    return outputs

def build_transformer_model():
    # A entrada do nosso modelo é uma sequência de 60 dias com N features
    inputs = keras.Input(shape=(SEQ_LENGTH, len(features)))

    # Empilhando vários blocos Transformer
    x = inputs
    for _ in range(NUM_BLOCKS):
        x = transformer_encoder(x)

    # Após passar pelos blocos, "achatamos" a sequência para uma única representação
    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    # Camadas densas para a classificação final
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(len(class_names), activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name="Financial_Transformer")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

transformer_model = build_transformer_model()
transformer_model.summary()

# ============================================================
# Parte 6: TREINAMENTO DO MODELO
# ============================================================
print("\nIniciando o treinamento do modelo Transformer...")

# Callback para parar o treino se a perda de validação não melhorar
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True, verbose=1
)

history = transformer_model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
)

print("Treinamento concluído.")

# ============================================================
# Parte 7: AVALIAÇÃO E PREVISÃO FINAL
# ============================================================
print("\nAvaliando o modelo e fazendo a previsão final...")

# Preparar os dados mais recentes para fazer a previsão
dados_recentes = data_featured.tail(SEQ_LENGTH)
dados_recentes_scaled = scaler.transform(dados_recentes)
X_pred = np.expand_dims(dados_recentes_scaled, 0) # Adiciona dimensão de lote

# Fazer a previsão
prediction = transformer_model.predict(X_pred)
score = prediction[0]
predicted_class_index = np.argmax(score)
predicted_class = class_names[predicted_class_index]

# --- Exibir o Resultado Final ---
print("\n" + "="*60)
print(f"🔮 PREVISÃO DO TRANSFORMER PARA OS PRÓXIMOS {FUTURE_DAYS} DIAS DE {TICKER}")
print("="*60)
for i, class_name in enumerate(class_names):
    print(f"Probabilidade de {class_name}: {100 * score[i]:.2f}%")
print("-" * 60)
print(f"🟢 TENDÊNCIA MAIS PROVÁVEL: {predicted_class}")
print("="*60)

Instalando e importando bibliotecas...
Bibliotecas carregadas com sucesso!

Definindo configurações globais.
Configurações definidas para o ativo: PETR4.SA

Baixando dados históricos para PETR4.SA...
Calculando indicadores técnicos...
Dados processados. Total de 2620 registros.

Preparando sequências e rótulos para o modelo...
Total de 2555 exemplos criados.
Dados de Treino: 2044 sequências
Dados de Validação: 511 sequências

Construindo a arquitetura do Transformer...


Model: "Financial_Transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 60, 5)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 5)     │     23,557 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 60, 5)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 60, 5)     │          0 │ input_layer[0][0… │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 60, 5)     │         10 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 60, 5)     │         49 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 60, 5)     │          0 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 60, 5)     │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 5)     │         10 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 5)     │     23,557 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 60, 5)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 60, 5)     │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 5)     │         10 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 60, 5)     │         49 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 60, 5)     │          0 │ sequential_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 60, 5)     │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 5)     │         10 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 5)     │     23,557 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat

 Total params: 102,699 (401.17 KB)

 Trainable params: 102,699 (401.17 KB)

 Non-trainable params: 0 (0.00 B)


Iniciando o treinamento do modelo Transformer...
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 54s 591ms/step - accuracy: 0.5146 - loss: 1.0556 - val_accuracy: 0.5362 - val_loss: 1.0065
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 36s 560ms/step - accuracy: 0.5259 - loss: 1.0136 - val_accuracy: 0.5362 - val_loss: 1.0039
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 43s 585ms/step - accuracy: 0.5307 - loss: 1.0131 - val_accuracy: 0.5362 - val_loss: 1.0055
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 39s 559ms/step - accuracy: 0.5572 - loss: 0.9810 - val_accuracy: 0.5362 - val_loss: 1.0026
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 36s 571ms/step - accuracy: 0.5299 - loss: 1.0078 - val_accuracy: 0.5362 - val_loss: 0.9991
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 36s 567ms/step - accuracy: 0.5442 - loss: 0.9963 - val_accuracy: 0.5362 - val_loss: 0.9997
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 42s 581ms/step - accuracy: 0.5407 - loss: 0.9956 - val_accuracy: 0.5362 - val_loss: 1.0118
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 39s 550ms